In [ ]:
from django_for_jupyter import init_django
init_django('loanpredictor')

In [ ]:
from loans.models import Loan
import pandas as pd
from sklearn import preprocessing

In [ ]:
qs = Loan.objects.ml_data()
qs = qs.values('state', 'term', 'no_emp','created_jobs','retained_jobs', 
                     'gross_appv', 'recession', 'secured_loan', 'gov_secured',
                     'is_rural', 'low_doc', 'new_business', 'econ_sector',
                     'inflation_on_loan','unemployment_on_loan', 'defaulted')

In [ ]:
df = pd.DataFrame(qs)
df.head()

In [ ]:
df.shape

In [ ]:
df = pd.DataFrame(qs)
lb = preprocessing.LabelBinarizer()
objList = df.select_dtypes(include = "bool").columns
for feature in objList:
    df[feature] = lb.fit_transform(df[feature])

df['gov_secured'] = df['gov_secured'].astype(float)
df['gross_appv'] = df['gross_appv'].astype(float)

In [ ]:
df.info()

In [ ]:
import xgboost as xgb
from skopt import BayesSearchCV
from skopt.space import Real, Integer
from sklearn.model_selection import train_test_split

In [ ]:

X, y = df.drop(['defaulted'], axis= 1), df['defaulted']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=.2)

bst = xgb.XGBClassifier() 
search_space = {
    'max_depth': Integer(3,10),
    'min_child_weight': Integer(0, 20),
    'learning_rate': Real(0.05, 1), 
    'subsample': Real(0.5, 1.0),
    'colsample_bytree': Real(0.5, 1.0),
    'colsample_bylevel': Real(0.5, 1.0),
    'colsample_bynode' : Real(0.5, 1.0),
    'alpha': Real(0.0, 10.0),
    'reg_lambda': Real(0.0, 10.0),
    'gamma': Real(0.0, 5),
    }


In [ ]:
opt = BayesSearchCV(bst, search_space, cv=5, n_iter=1, scoring='recall', random_state=8) 

In [ ]:
opt.fit(X_train, y_train)

In [ ]:
xgb_params = opt.best_params_

In [ ]:
xgb_params

In [ ]:
model = xgb.XGBClassifier(**xgb_params)
model.fit(X_train, y_train)

In [ ]:
import pickle
binary_data = pickle.dumps(model)

In [ ]:
binary_data

In [1]:
from django_for_jupyter import init_django
init_django('loanpredictor')
from ml.utlis import train_xgboost_model

In [2]:
import pickle
from datetime import date, datetime

from django.core.files.base import File
from django.conf import settings
from django.apps import apps
from ml.models import Model

import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import recall_score, precision_score, accuracy_score
from sklearn.preprocessing import LabelBinarizer
import xgboost as xgb
from skopt import BayesSearchCV
from skopt.space import Real, Integer

In [8]:
np.version.version

'1.23.5'

In [4]:
from loans.models import Loan
qs = Loan.objects.ml_data()
qs = qs.values('state', 'term', 'no_emp','created_jobs','retained_jobs', 
                    'gross_appv', 'recession', 'secured_loan', 'gov_secured',
                    'is_rural', 'low_doc', 'new_business', 'econ_sector',
                    'inflation_on_loan','unemployment_on_loan', 'defaulted')

In [6]:
df = pd.DataFrame(qs)
lb = LabelBinarizer()
objList = df.select_dtypes(include = "bool").columns
for feature in objList:
    df[feature] = lb.fit_transform(df[feature])
objList = df.select_dtypes(include = "int32").columns
for feature in objList:
    df[feature] = df[feature].astype(int)
objList = df.select_dtypes(include = "int64").columns
for feature in objList:
    df[feature] = df[feature].astype(int)
df['gov_secured'] = df['gov_secured'].astype(float)
df['gross_appv'] = df['gross_appv'].astype(float)
df.head()

<bound method NDFrame.head of         state  term  no_emp  created_jobs  retained_jobs  gross_appv   
0          26     6      30             0             30    210000.0  \
1          24     3      10             0             10      5000.0   
2          17     5       4             8             12    100000.0   
3          19     3      20             0             20     50000.0   
4          30     6      18            15             18    100000.0   
...       ...   ...     ...           ...            ...         ...   
677446     14    60      11             0              0     85000.0   
677447      5   318       3             0              0     52500.0   
677448     35    84      34             0              0    150000.0   
677449     34   162      20             0              0    100000.0   
677450     38   246       0             0              0     27200.0   

        recession  secured_loan  gov_secured  is_rural  low_doc  new_business   
0               0       

In [7]:
X, y = df.drop(['defaulted'], axis= 1), df['defaulted']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=.2)

In [8]:
bst = xgb.XGBClassifier() 
#Adjust Search Grid
search_space = {
    'max_depth': Integer(3,10),
    'min_child_weight': Integer(0, 20),
    'learning_rate': Real(0.05, 1), 
    'subsample': Real(0.5, 1.0),
    'colsample_bytree': Real(0.5, 1.0),
    'colsample_bylevel': Real(0.5, 1.0),
    'colsample_bynode' : Real(0.5, 1.0),
    'alpha': Real(0.0, 10.0),
    'reg_lambda': Real(0.0, 10.0),
    'gamma': Real(0.0, 5),
    }
opt = BayesSearchCV(bst, search_space, cv=5, n_iter=1, scoring='recall')

In [9]:
opt.fit(X_train.to_numpy(), y_train.ravel())

BayesSearchCV(cv=5,
              estimator=XGBClassifier(base_score=None, booster=None,
                                      callbacks=None, colsample_bylevel=None,
                                      colsample_bynode=None,
                                      colsample_bytree=None,
                                      early_stopping_rounds=None,
                                      enable_categorical=False,
                                      eval_metric=None, feature_types=None,
                                      gamma=None, gpu_id=None, grow_policy=None,
                                      importance_type=None,
                                      interaction_constraints=None,
                                      learning_rate=None...
                             'learning_rate': Real(low=0.05, high=1, prior='uniform', transform='normalize'),
                             'max_depth': Integer(low=3, high=10, prior='uniform', transform='normalize'),
                             'min_child_weight': Integer(low=0, high=20, prior='uniform', transform='normalize'),
                             'reg_lambda': Real(low=0.0, high=10.0, prior='uniform', transform='normalize'),
                             'subsample': Real(low=0.5, high=1.0, prior='uniform', transform='normalize')})

In [ ]:
xgb_params = opt.best_params_
model = xgb.XGBClassifier(**xgb_params)
model.fit(X_train, y_train)
y_pred = model.predict(X_test)
recall = recall_score(y_pred, y_test, pos_label=1)
precision = precision_score(y_pred, y_test, pos_label=1)
accuracy = accuracy_score(y_pred, y_test)
acc_label = round((100* accuracy),2)

In [ ]:
opt.fit(X_train, y_train)


In [4]:
from django_for_jupyter import init_django
init_django('loanpredictor')
from ml.utlis import train_xgboost_model
train_xgboost_model(cv=5, n_iter=1, scoring='recall', verbose=True)

677451 loans were loaded for training.
Bayes Search was performed successfully
Model was trained successfully
New model: model-93.57-2023-05-06, successfully created


In [1]:
from django_for_jupyter import init_django
init_django('loanpredictor')


In [4]:
from loans.models import Loan
from dateutil.relativedelta import relativedelta
from datetime import datetime, date


In [8]:
queryset = Loan.objects.filter(active=True)
now = date.today()
TIME_AFTER_MATURITY_ACTIVE = 12
updated = 0
for obj in queryset:
    if obj.approval_date:
        active_frame = obj.maturity_date + relativedelta(months=+TIME_AFTER_MATURITY_ACTIVE)
        if active_frame < now or obj.defaulted:
            obj.active = False
            updated += 1

Loan.objects.bulk_update(queryset, ['active'])
print(f"Updated {updated} movie idx fields")



Updated 63755 movie idx fields


In [ ]:
import time
from datetime import datetime
start_time = datetime.now() 

time_elapsed = datetime.now() - start_time 
type(time_elapsed)
print(time_elapsed)

In [ ]:
from ml.models import Model

In [ ]:
qs = Model.objects.all().exclude(model_id=3)

In [ ]:
from django.utils import timezone

In [ ]:
qs.update(active=False, active_update_timestamp=timezone.now())

In [ ]:
import pickle
raw_model = Model.objects.filter(active=True)
if raw_model.exists():
    if len(raw_model) == 1:
        model = pickle.loads(raw_model[0].model)
    else:
        raw_model = raw_model.latest('created')
        model = pickle.loads(raw_model[0].model)        
else:
    raw_model = Model.objects.latest('created')
    model = pickle.loads(raw_model[0].model)

In [ ]:
raw_model = Model.objects.filter(active=False)
print(raw_model)

In [ ]:
raw_model.latest('created')

In [ ]:
from django.apps import apps
Loan = apps.get_model('loans', 'Loan')

In [ ]:
print(ctype)

In [ ]:
Loan.objects.all()

In [ ]:
from ml.utlis import load_model
import pandas as pd

In [ ]:
from django_for_jupyter import init_django
init_django('loanpredictor')
from ml.utlis import prepare_data

In [ ]:
model = load_model()

In [ ]:
from loans.models import Loan
qs = Loan.objects.latest('last_updated')
loan_data = {'state': qs.state, 
                'term': qs.term, 
                'no_emp':qs.no_emp,
                'created_jobs':qs.created_jobs,
                'retained_jobs':qs.retained_jobs, 
                'gross_appv':qs.gross_appv, 
                'recession': qs.recession, 
                'secured_loan':qs.secured_loan, 
                'gov_secured':qs.gov_secured,
                'is_rural':qs.is_rural, 
                'low_doc':qs.low_doc, 
                'new_business':qs.new_business, 
                'econ_sector':qs.econ_sector,
                'inflation_on_loan':qs.inflation_on_loan,
                'unemployment_on_loan':qs.unemployment_on_loan}
loan_data

In [ ]:
df = pd.DataFrame(loan_data, index=[0])
df.head()

In [ ]:
df = prepare_data(df, verbose=False)
df.head()

In [ ]:
risk = type(model.predict(df)[0])
risk

In [ ]:
from ml.utlis import optimize_loan_parameters

In [ ]:
optimized = optimize_loan_parameters(df, bst=model)
optimized

In [ ]:
model.predict_proba(df)[0]

In [ ]:
from loans.models import Loan

In [ ]:
Loan.objects.prefetch_related('riskassessement').all()[:100]